In [5]:
!pip install openai
!pip install pymupdf # Install PyMuPDF which includes the correct fitz module.
!pip install -U pip setuptools wheel
!pip uninstall -U spacy
!pip install -U spacy==3.7.6
import spacy
import os
import pandas as pd
nlp = spacy.load("en_core_web_sm")


Usage:   
  pip uninstall [options] <package> ...
  pip uninstall [options] -r <requirements file> ...

no such option: -U


In [23]:
from pypdf import PdfReader
import os
start='abstract'
end='references'

start_page=0
main_dir="/content/Docs/"
def find_index(text,target):
    try:
        return text.index(target)
    except:
        return -1

for files in os.listdir(main_dir):
    passage=""
    found=False
    path=main_dir+files
    if files[-3:]!="pdf":
        continue
    reader = PdfReader(path)
    number_of_pages = len(reader.pages)
    for page_num in range(number_of_pages):
        page = reader.pages[page_num]
        text = [texts.lower() for texts in page.extract_text().split()]
        start_idx=find_index(text,start)+1
        end_idx=find_index(text,end)
        if found:
            if start_idx == -1 and end_idx==-1:
                passage += " ".join(text)
            elif end_idx != -1 and start_idx == -1:
                passage += " ".join(text[:end_idx])
            else:
                passage += " ".join(text[start_idx:end_idx])
        elif not found:
            if start_idx != -1 and end_idx != -1:
                found=True
                passage += " ".join(text[start_idx:end_idx])
            elif start_idx == -1 and end_idx != -1:
                found=True
                passage += " ".join(text[:end_idx])
            elif start_idx == -1 and end_idx == -1:
                continue
            else:
                found=True
                passage += " ".join(text[start_idx:])
    with open(main_dir+"text/"+files[:-3]+".txt", 'w') as f:
        f.write(passage)




direct pdf w/ llm

In [1]:
API_KEY=''
modeltype= 'gpt-4-turbo-preview'
main_dir="Docs/"

In [6]:
!pwd

/Users/rishikasrinivas/KnowledgeGraphMentalHealth


In [ ]:
#pdf llm
import os
from openai import OpenAI

client= OpenAI( api_key= API_KEY)

for filename in os.listdir(main_dir):
    print(filename[-3:])
    if filename[-3:] != 'pdf':
        continue
    filename= main_dir + filename
    assistant_name= 'Clinicial Assistant'
    
    instructions=  """
      json: You are to extract important entities, their definitions, overarching ideas,  and their relationships with one another from this paper. Don't copy the sentence
      word for word and split it up. Do not take all random relationships. Identify relationships that can help clinicians with diagnosis. GET THE 
      MAIN IDEA!!. Will/May need to look across sections.
      
      Understand the sentence and determine the relationships between the subject and objects
    
      DO NOT use the word 'Depression' ALONE as a subject or object. Each block of text emphasizes different types of depression and you must include the type of depression the relationship is referring to.
      If the text is referring to all types of depression for that relationship, label it as "All Depression"
      The idea is that these relationships can be pieced together into a knowledge graph that a clinician can trace. Keep this in mind as you extract.
    
      Do not use any relationships other than these: 
        1. treatment:  IPT --> treatment for -> depression
        2. symptom: sadness -> symptom of -> depression
        3. example: medA --> example of -> antidepressant
        4. not effective: medA -> not effective for --> depression
        5. effective: medA -> effective for -> depression
        6. more effective: medA -> more effective than -> medB
        7. less effective: medA -> less effective than -> medB
        8. side effects: sneezing --> side effect of -> medA
        9. treatment:  meditate --> treatment to -> reduce depression
        10. Same efficacy: medA -> Same efficacy as -> no meds
        11. Recommended first-line treatment: TreatA --> Recommended first-line treatment for --> illnesses
        12. High Efficiency: MedA -> High Efficiency for -> DiseaseA
        13. Less commonly used: MedB -> Less commonly used for -> Depression
        14. correlated: DiseaseA -> correlated with -> ProblemA
        15. More common than ... in ... : DiseaseA -> More common than -> DiseaseB -> in -> younger patients
        16. Concurrent comorbidity: Disease A -> Concurrent comorbidity -> IllnessC
        17. Findings: DiseaseA -> findings -> faster treatment in young people
    
    
    
      Example:  As shown in Table 2, remission rates were significantly lower in patients with anxious depression,
      according to both the HAM-D criterion (22.2% versus 33.4%) and the QIDS-SR criterion (27.5% versus 38.9%). 
      Response rates were also significantly lower for patients with anxious depression (41.7% versus 52.8%). "
    
      Gives the rel(s): Anxious Depression -> finding -> low remission rates  and Anxious Depression -> finding -> low response rates 
      
      Present 1 dataframe in an easily readable and visually appealing format.
      The dataframe should explain the relationships between all subjects.
      It should have a 4 headers, subj, rel, obj, reference. Reference should hold the exact sentence(s) (in quotes) that you used to get the relationship
      If you have object2 as a header, you must have a non-empty rel2 as a header. If thats not possible place the relationship in the next row
    
    
      return a pandas dataframe structure called paper1
      Return a form of a dictionary of lists.
      Avoid entities and relationships that are more than a few words. If you need to list multiple drug names or treatments, list them in different rows


    """
    
    assistant= client.beta.assistants.create(
    name= assistant_name,
    instructions= instructions,
    model= modeltype,
    tools= [{"type":"file_search"}])
    
    message_file= client.files.create(
    file=open(filename, "rb"),
    purpose="assistants")
    thread= client.beta.threads.create(
    messages=[
        {
            "role":"user",
            "content": instructions,
            "attachments":[
                {"file_id": message_file.id,
                "tools":[{"type":"file_search"}]}
            ],
    
        }
    ]
                 )
    
    run= client.beta.threads.runs.create_and_poll(thread_id= thread.id, assistant_id= assistant.id)
    
    messages= list(client.beta.threads.messages.list(thread_id=thread.id, run_id= run.id))
    message_content= messages[0].content[0].text
    annotations= message_content.annotations
    citations=[]
    for index, annotation in enumerate(annotations):
              message_content.valye= message_content.value .replace(annotation.text,""). replace(".", "")
              if file_citation :=getattr(annotations, "file_citation", None):
                  cited_file= client.files.retrieve(file_citation.file.id)
                  citations.append(f"[{index}] {cited_file.filename}")
    cand_name= message_content.value.replace(".", ""). strip()
    print(cand_name)

In [87]:
data1 = {
    "subj": [
        "Mirtazapine", "Venlafaxine", "Sertraline",
        "Escitalopram", "Reboxetine", "Duloxetine", "Fluoxetine",
        "Fluvoxamine", "Paroxetine", "Bupropion", "Citalopram",
        "Milnacipran"
    ],
    "rel": [
        "more effective than", "more effective than", "more effective than",
        "more effective than", "less effective than", "less well tolerated than",
        "less effective than", "less well tolerated than", "less well tolerated than",
        "better tolerated than", "more effective than", "more effective than"
    ],
    "object1": [
        "Fluoxetine", "Fluoxetine", "Duloxetine and Fluvoxamine",
        "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", "all other 11 antidepressants",
        "Escitalopram and Sertraline", "many other antidepressants", "Citalopram, Escitalopram, Sertraline",
        "Escitalopram and Sertraline", "Duloxetine, Fluvoxamine, Paroxetine, Reboxetine",
        "Fluoxetine", "Fluvoxamine"
    ]
}
data2 = {
    "subj": ["MAOIs", "TCAs", "ECT", "SSRIs", "Nefazodone", "Trazodone", "Vortioxetine", "Ketamine", "MAOIs", "TCAs", "Bupropion", "MAOIs", "SSRIs", "SSRIs", "Escitalopram", "Duloxetine", "Venlafaxine"],
    "rel": ["example of", "effective for", "treatment for", "less effective than", "side effect of", "side effect of", "effective for", "effective for", "not effective for", "more effective than", "side effects", "treatment option when", "effective for", "side effects", "effective for", "effective for", "more effective than"],
    "object1": ["antidepressant", "depression with atypical features", "depression", "TCAs for hospitalized patients", "hepatotoxicity", "somnolence", "depression", "resistant depression and acute suicidal ideation", "depression resistant to SSRIs and SNRIs", "SSRIs for hospitalized patients", "minimal weight gain or even weight loss; risk for seizures", "ECT is contraindicated", "major depressive disorder with high levels of anxiety", "several minor side effects including sexual dysfunction", "major depressive disorder", "major depressive disorder", "SSRIs"],
    "rel2": ["side effects", "treatment option when", "", "no difference in", "", "", "", "", "", "", "", "", "", "", "", "", ""],
    "object2": ["numerous including hepatotoxicity and hypertensive crises", "other pharmacotherapies have not succeeded", "", "outpatients less severely ill", "", "", "", "", "", "", "", "", "", "", "", "", ""]
}

data3 = {
    "subj": ["Anxious Depression", "Citalopram", "Citalopram + Benzodiazepine", "Bupropion (SR)", "Sertraline", "Venlafaxine (XR)", "Citalopram + Bupropion", "Citalopram + Buspirone", "Side Effects"],
    "rel": ["less likely to", "treatment for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "greater in"],
    "object1": ["achieve remission than Nonanxious Depression", "Depression", "Anxious Depression", "Nonanxious Depression", "Nonanxious Depression", "Nonanxious Depression", "Anxious Depression", "Anxious Depression", "Anxious Depression"],
    "rel2": ["", "", "leads to", "", "", "", "", "", "associated with"],
    "object2": ["", "", "remission of symptoms", "", "", "", "", "", "anxious features in Depression"]
}

d1=pd.DataFrame(data1)
d2=pd.DataFrame(data2)
d3=pd.DataFrame(data3)
pd.concat([d1,d2,d3]).reset_index().drop(columns=['index']).to_csv('pdfLLM.csv')

In [70]:
data = {"subj": ["Venlafaxine", "Escitalopram", "Mirtazapine", "Reboxetine", "Citalopram", "Sertraline", "Duloxetine", "Fluoxetine", "Paroxetine"],
        "rel": ["effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for"],
        "object1": ["Depression", "Depression", "Depression", "Depression", "Depression", "Depression", "Depression", "Depression", "Depression"],
        "rel2": ["more effective than", "less effective than", "more effective than", "more effective than",
                 "less effective than", "more effective than", "less effective than", "more effective than", "more effective than"],
         "object2": ["Bupropion", "Duloxetine", "Fluoxetine", "Fluoxetine", "Sertraline",
                     "Fluoxetine", "Escitalopram", "Milnacipran", "Fluoxetine"],
        "rel3": ["side effect of", "side effect of", "side effect of", "side effect of", "side effect of", "side effect of", "side effect of", "side effect of", "side effect of"],
        "object3": ["N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A"]}

In [71]:
pd.DataFrame(data)

,subj,rel,object1,rel2,object2,rel3,object3
0,Venlafaxine,effective for,Depression,more effective than,Bupropion,side effect of,N/A
1,Escitalopram,effective for,Depression,less effective than,Duloxetine,side effect of,N/A
2,Mirtazapine,effective for,Depression,more effective than,Fluoxetine,side effect of,N/A
3,Reboxetine,effective for,Depression,more effective than,Fluoxetine,side effect of,N/A
4,Citalopram,effective for,Depression,less effective than,Sertraline,side effect of,N/A
5,Sertraline,effective for,Depression,more effective than,Fluoxetine,side effect of,N/A
6,Duloxetine,effective for,Depression,less effective than,Escitalopram,side effect of,N/A
7,Fluoxetine,effective for,Depression,more effective than,Milnacipran,side effect of,N/A
8,Paroxetine,effective for,Depression,more effective than,Fluoxetine,side effect of,N/A


In [8]:
!pwd

/Users/rishikasrinivas/KnowledgeGraphMentalHealth


nlp processing w/ llm

In [6]:
import fitz
import os
import pandas as pd
texts=[]
# Opening the PDF file
skip=6
for doc in os.listdir(main_dir):
    doc='fava-et-al-2008-difference-in-treatment-outcome-in-outpatients-with-anxious-versus-nonanxious-depression-a-star_d-report.pdf'
    print(doc[-3:])
    if doc[-3:] != 'pdf':
        continue
    doc=fitz.open(main_dir+doc)
    # Extracting text from all pages
    all_text = []
    for page_num in range(0,len(doc),skip):
        tex=""
        for p in range(page_num, page_num+skip):
          if p >= len(doc):
            break
          page = doc[p]
          text = page.get_text().replace('\n', ' ').lower()
          docs=nlp(text)
          text=" ".join([token.lemma_ for token in docs])
          tex += text
        all_text.append([page_num + 1, tex])
        df = pd.DataFrame(all_text, columns=['Page', 'Text'])
        for text in df['Text'].to_list():
            if 'References' in text:
                break
            texts.append(text)
    break

pdf


In [75]:
len(texts)

3

In [7]:
from openai import OpenAI
client= OpenAI( api_key= API_KEY)

import json
import pandas as pd
df=pd.DataFrame()
for t in texts:
  prompt_llm= """
  json: GET THE 
  MAIN IDEA!! You are to extract important entities, their definitions, overarching ideas,  and their relationships with one another from this paper. Don't copy the sentence
  word for word and split it up. Do not take all random relationships. Identify relationships that can help clinicians with diagnosis. . Will/May need to look across sections. Refrain from forming relationships unless text explicitly highlights or suggests that.

  For example, do not say a medicine is/isn't effective unless the text explicity says so
  
  Understand the sentence and determine the relationships between the subject and objects

  DO NOT use the word 'Depression' ALONE as a subject or object. Each block of text emphasizes different types of depression and you must include the type of depression the relationship is referring to.
  If the text is referring to all types of depression for that relationship, label it as "All Depression"
  The idea is that these relationships can be pieced together into a knowledge graph that a clinician can trace. Keep this in mind as you extract.

  Do not use any relationships other than these: 
    1. treatment:  IPT --> treatment for -> depression
    2. symptom: sadness -> symptom of -> depression
    3. example: medA --> example of -> antidepressant
    4. not effective: medA -> not effective for --> depression
    5. effective: medA -> effective for -> depression
    6. more effective: medA -> more effective than -> medB
    7. less effective: medA -> less effective than -> medB
    8. side effects: sneezing --> side effect of -> medA
    9. treatment:  meditate --> treatment to -> reduce depression
    10. Same efficacy: medA -> Same efficacy as -> no meds
    11. Recommended first-line treatment: TreatA --> Recommended first-line treatment for --> illnesses
    12. High Efficiency: MedA -> High Efficiency for -> DiseaseA
    13. Less commonly used: MedB -> Less commonly used for -> Depression
    14. correlated: DiseaseA -> correlated with -> ProblemA
    15. More common than ... in ... : DiseaseA -> More common than -> DiseaseB -> in -> younger patients
    16. Concurrent comorbidity: Disease A -> Concurrent comorbidity -> IllnessC
    17. Findings: DiseaseA -> findings -> faster treatment in young people
    18. predictor of: use of medA -> predictor of -> diseaseB
    19. Associateed with: Disease A -> associted with -> diases B
    20. definition: therapyA -> definition -> defA
    21: Correlation: ilnnessA -> correlation -> ilnessB
    22. Most common treatment: DiseaseA -> Most common treatment -> MedA
    23. Second most common treatment: DiseaseA - Second most common treatment -> MedA
    24. Use: MedA -> use -> decreasing symptomA or medA -> used on -> patients with problem B
    25. Properties: medC -> properties -> reduces this symptom by inihibiting that neuron
    26: form of: IllnessA -> form of -> CategroyA
    26. goal: MedA -> goal -> reduce anger
    27: requirements: treatmentA -> requirements -> patient under 23
    
    



  Example:  As shown in Table 2, remission rates were significantly lower in patients with anxious depression,
  according to both the HAM-D criterion (22.2% versus 33.4%) and the QIDS-SR criterion (27.5% versus 38.9%). 
  Response rates were also significantly lower for patients with anxious depression (41.7% versus 52.8%). "

  Gives the rel(s): Anxious Depression -> finding -> low remission rates  and Anxious Depression -> finding -> low response rates 
  
  Present 1 dataframe in an easily readable and visually appealing format.
  The dataframe should explain the relationships between all subjects.
  It should have a 4 headers, subj, rel, obj, reference. Reference should hold the exact sentence(s) (in quotes) that you used to get the relationship
  If you have object2 as a header, you must have a non-empty rel2 as a header. If thats not possible place the relationship in the next row


  return a pandas dataframe structure called paper1
  Return a form of a dictionary of lists.
  Avoid entities and relationships that are more than a few words. If you need to list multiple drug names or treatments, list them in different rows


  """

  prompt=f"Using this information: {t}, answer the following question: {prompt_llm}"

  chat_completion=client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": prompt_llm,
          },
          {
              "role": "user",
              "content": prompt,
          }
      ],
  model= modeltype,
    )
  resp=chat_completion.choices[0].message.content
  start_idx=resp.find("data = ")
  if resp[start_idx+7] == '[':
    end_idx=resp[start_idx:].find(']')
  else:
    end_idx=resp[start_idx:].find('}')
  resp=resp[start_idx+7:start_idx+end_idx+1]
  df=pd.concat([df, pd.DataFrame(eval(resp))])
  print(eval(resp))
  print("\n\n\n")






{'subj': ['Anxious Depression', 'Anxious Depression', 'Anxious Depression', 'Anxious Depression', 'Anxious Depression', 'Anxious Depression', 'Major Depressive Disorder', 'Anxious Depression', 'Anxious Depression', 'Anxious Depression', 'Anxious Depression'], 'rel': ['findings', 'findings', 'associated with', 'correlated with', 'more likely to have', 'finding', 'prevalence', 'less likely to achieve', 'less likely to achieve', 'finding', 'finding'], 'obj': ['low remission rates', 'low response rates', 'poorer acute outcomes than Nonanxious Depression', 'greater severity of depressive illness and functional impairment', 'increased risk of suicidality', 'greater illness chronicity', 'comorbidity rate of 40%-50% with anxiety disorders', 'remission in Level 2 with switch or augmentation', 'remission in Level 2 with switch or augmentation', 'greater side effect frequency, intensity, and burden', 'more serious adverse events'], 'reference': ['Remission rates were significantly lower in patien

In [8]:
df

,subj,rel,obj,reference
0,Anxious Depression,findings,low remission rates,Remission rates were significantly lower in pa...
1,Anxious Depression,findings,low response rates,Remission rates were significantly lower in pa...
2,Anxious Depression,associated with,poorer acute outcomes than Nonanxious Depression,Anxious depression is associated with poorer a...
3,Anxious Depression,correlated with,greater severity of depressive illness and fun...,In patients with high levels of anxiety accomp...
4,Anxious Depression,more likely to have,increased risk of suicidality,An increase risk of suicidality has been repor...
5,Anxious Depression,finding,greater illness chronicity,Greater illness chronicity has been reported i...
6,Major Depressive Disorder,prevalence,comorbidity rate of 40%-50% with anxiety disor...,"In outpatient with major depression, early stu..."
7,Anxious Depression,less likely to achieve,remission in Level 2 with switch or augmentation,Patients with anxious depression fare signific...
8,Anxious Depression,less likely to achieve,remission in Level 2 with switch or augmentation,Patients with anxious depression fare signific...
9,Anxious Depression,finding,"greater side effect frequency, intensity, and ...","Rating of side effect frequency, intensity, an..."


In [23]:
data1= [
    # Defining the treatment for depression
    {"subj": "Selective Serotonin Reuptake Inhibitors (SSRIs)", "rel": "treatment for", "obj": "depression"},
    {"subj": "Electroconvulsive Therapy", "rel": "most effective somatic therapy for", "obj": "depression in specific situations"},
    {"subj": "Combination of medication, psychotherapy, and somatic therapy", "rel": "most effective way to manage", "obj": "resistant depression"},
    # Defining examples of antidepressants
    {"subj": "Fluoxetine, Sertraline, Paroxetine", "rel": "example of", "obj": "SSRIs"},
    {"subj": "Venlafaxine, Desvenlafaxine, Duloxetine", "rel": "example of", "obj": "SNRIs"},
    # Defining effectiveness of treatments
    {"subj": "TCA", "rel": "more effective than", "obj": "SSRIs for hospitalized patients"},
    {"subj": "MAOIs", "rel": "effective for", "obj": "depression with atypical features"},
    # Side effects of medications
    {"subj": "SSRIs", "rel": "side effects of", "obj": "sexual and digestive issues, irritability, anxiety, insomnia"},
    {"subj": "MAOIs", "rel": "side effects of", "obj": "hepatotoxicity, hypertensive crisis"},
    # Other antidepressants
    {"subj": "Bupropion", "rel": "less likely to cause", "obj": "weight gain"},
    {"subj": "Bupropion", "rel": "more likely than SSRIs", "obj": "to improve symptoms of fatigue and sleepiness"},
]




data2 = {
    "subj": ["Pharmacotherapy", "SSRI Antidepressants", "Psychotherapy", "ECT", "MAOIs", "TCAs", "SNRIs", "Fluoxetine", "Venlafaxine"],
    "rel": ["treatment for", "example of", "treatment for", "treatment for", "less effective than", "more effective than", "recommended after", "effective for", "effective for"],
    "obj": ["Acute phase depression", "Antidepressant", "Depression", "Certain depression cases", "Other antidepressants in specific conditions", "SSRIs in hospitalized patients", "SSRIs", "Depression", "Depression"]
}




data3= {
  "subj": [
    "Mirtazapine", "Mirtazapine", "Vilazodone", "Vortioxetine", "Vortioxetine", "Citalopram", "Escitalopram", "Fluoxetine", 
    "Sertraline", "Vortioxetine", "Amitriptyline", "Clomipramine", "Duloxetine", "Fluvoxamine", "Trazodone", "Venlafaxine", 
    "Nausea and vomiting", "Sexual dysfunction", "Ketamine", "Ketamine", "Esketamine", 
    "CBT", "IPT", "CBT", "PEI", "MBCT", "CBASP"
  ],
  "rel": [
    "example of", "effective for", "example of", "effective for", "treatment to", "more tolerable than", "more tolerable than", "more tolerable than", 
    "more tolerable than", "more tolerable than", "less tolerable than", "less tolerable than", "less tolerable than", "less tolerable than", "less tolerable than", 
    "less tolerable than", "side effect of", "side effect of", "effective for", "has side effects of", "approved for", 
    "effective for", "effective for", "recommend for", "recommend for", "recommend for", "recommend for"
  ],
  "obj": [
    "Antidepressant", "Depression", "Antidepressant", "Depression", "Reduce depression-related cognitive impairment", "Other antidepressants", "Other antidepressants", "Other antidepressants",
    "Other antidepressants", "Other antidepressants", "Amitriptyline", "Clomipramine", "Duloxetine", "Fluvoxamine", "Trazodone", "Venlafaxine", 
    "Antidepressants", "Antidepressants", "Treatment-resistant depression", "Dizziness, neurotoxicity, cognitive dysfunction, blurred vision, psychosis, dissociation, urological dysfunction, restlessness, headache, cardiovascular symptoms", 
    "Treatment-resistant depression", "Depression", "Depression", "Depression", "Depression", "Depression", "Chronic depression"
  ]
}




data4 = [
    {"subj": "Selective Serotonin Reuptake Inhibitors (SSRIs)", "rel": "effective for", "obj": "Depression"},
    {"subj": "Electroconvulsive therapy (ECT)", "rel": "most effective somatic therapy for", "obj": "Depression in specific situations"},
    {"subj": "Fluoxetine", "rel": "example of", "obj": "SSRI"},
    {"subj": "Sertraline", "rel": "example of", "obj": "SSRI"},
    {"subj": "Citalopram", "rel": "example of", "obj": "SSRI"},
    {"subj": "SSRIs", "rel": "side effects of", "obj": "sexual and digestive issues, irritability, anxiety, insomnia, headaches"},
    {"subj": "Monoamine Oxidase Inhibitors (MAOIs)", "rel": "less effective than", "obj": "Tricyclic Antidepressants (TCAs)"},
    {"subj": "Tricyclic Antidepressants (TCAs)", "rel": "more effective than", "obj": "SSRIs for severe MDD requiring hospitalization"},
    {"subj": "Venlafaxine", "rel": "example of", "obj": "SNRI"},
    {"subj": "SSRIs", "rel": "first-line treatment for", "obj": "Major Depression"},
    {"subj": "Bupropion", "rel": "side effects of", "obj": "minimal weight gain or weight loss, improves symptoms of fatigue and sleepiness"},
    {"subj": "MAOIs", "rel": "treatment for", "obj": "depression with atypical features"},
    {"subj": "Trazodone", "rel": "used as", "obj": "sedative-hypnotic rather than an antidepressant in contemporary practice"}
]




data5 = {
    "subj": ["Mirtazapine", "Vilazodone", "Vortioxetine", "SSRIs", "Ketamine",
             "Esketamine", "MAOIs", "TCAs", "Fluoxetine", "SSRIs", "SNRIs", "Trazodone",
             "Bupropion", "Vortioxetine", "Vilazodone", "Ketamine", "Esketamine", "CBT", "IPT"],
    "rel": ["example of", "less sexual dysfunction than", "effective for", "as effective as", "effective for",
            "effective for", "disadvantage", "disadvantage", "side effects of", "side effects of", "side effects of",
            "side effects of", "side effects of", "side effects of", "side effects of", "side effects of", "risk of abuse",
            "treatment for", "treatment for"],
    "obj": ["Antidepressant", "SSRIs", "depression-related cognitive impairment", "Mirtazapine", "resistant depression",
            "treatment-resistant depression", "drug interaction, dietary restriction", "cardiovascular toxicity and anticholinergic side effects",
            "sexual dysfunction, loss of appetite, vomiting, nausea, irritability, anxiety, insomnia, headache", "paroxetine - high rate of sexual dysfunction",
            "high rate of nausea, vomiting, sexual dysfunction", "somnolence", "risk for seizure", "nausea", "diarrhea and nausea",
            "nausea, vomiting, possible neurotoxicity and drug dependence", "Esketamine", "depression", "depression"]
}




data6 = {
    "subj": [
        "CBT", "Behavioral Activation", "Acceptance and Commitment Therapy", "CCBT", 
        "Antidepressants", "Schema Therapy", "IPT", "PST", "Marital and Family Therapy", 
        "ST", "Psychodynamic Therapy", "Group Therapy", "MBCT", "Psycho-education", 
        "Physical Exercise", "ECT", "RTMS", "TDCS"
    ],
    "rel": [
        "treatment for", "example of", "example of", "example of",
        "treatment to", "effective for", "treatment for", "treatment for", 
        "treatment for", "treatment for", "effective for", "effective for", 
        "treatment to", "treatment to", "treatment to", "treatment for", 
        "treatment for", "treatment for"
    ],
    "obj": [
        "Depression", "CBT technique", "form of CBT", "form of CBT",
        "Reduce Depression", "Depression", "Depression", "Depression", 
        "Some aspects of Depression", "Depression", "Acute phase of MDD", "Specific MDD", 
        "Reduce Antidepressant Treatment", "Improve Clinical Course", "Alleviate Symptoms", 
        "Resistant Depression", "Depression", "Depressive Symptoms"
    ]
}




data7 = {
    'subj': ['SSRIs', 'MAOIs', 'TCAs', 'SSRIs', 'SSRIs', 'Venlafaxine', 'Duloxetine', 'Reboxetine', 'Bupropion', 'Trazodone', 'Nefazodone', 'Bupropion', 'MAOIs', 'Electroconvulsive therapy', 'Pharmacotherapy', 'Psychotherapy', 'Electroconvulsive therapy'],
    'rel': ['treatment for', 'treatment for', 'effective for', 'example of', 'less effective than', 'comparable to', 'comparable to', 'less effective than', 'comparable to', 'less common use', 'associates with', 'more likely than', 'more effective than', 'effective for', 'includes', 'includes', 'most effective somatic therapy for'],
    'obj': ['Depression', 'Depression', 'Severe Major Depression', 'Antidepressant', 'TCAs', 'SSRIs', 'SSRIs', 'SSRIs', 'SSRIs in low to moderate anxiety MDD', 'Antidepressant usage', 'idiosyncratic hepatotoxicity', 'SSRIs to improve symptoms of fatigue and sleepiness', 'Other antidepressants in certain cases', 'Depression in specific situations', 'SSRIs', 'Depression-focused therapies', 'Depression in specific situations']
}




data8 = [
  {"subj": "Mirtazapine", "rel": "effective for", "obj": "depression"},
  {"subj": "SSRIs", "rel": "effective for", "obj": "depression"},
  {"subj": "Vilazodone", "rel": "less sexual dysfunction than", "obj": "SSRIs"},
  {"subj": "Vortioxetine", "rel": "treatment for", "obj": "depression-related cognitive impairment"},
  {"subj": "Ketamine", "rel": "effective for", "obj": "resistant depression"},
  {"subj": "Esketamine", "rel": "approved for", "obj": "treatment-resistant depression"},
  {"subj": "Antidepressants", "rel": "not effective for", "obj": "every patient"},
  {"subj": "Citalopram", "rel": "more tolerable than", "obj": "Amitriptyline, Clomipramine"},
  {"subj": "Nausea", "rel": "side effect of", "obj": "SSRIs, SNRIs"},
  {"subj": "Sexual dysfunction", "rel": "side effect of", "obj": "SSRIs, SNRIs"},
  {"subj": "CBT", "rel": "treatment for", "obj": "depression"},
  {"subj": "IPT", "rel": "treatment for", "obj": "depression"}
]




data9 = {
    "subj": [
        "CBT", "Behavioral Activation", "Acceptance and Commitment Therapy",
        "CCBT", "Schema Therapy", "IPT", "Problem-Solve Therapy",
        "Marital and Family Therapy", "Supportive Therapy", "Psychodynamic Therapy",
        "Group Therapy", "MBCT", "Psycho-Education", "Physical Exercise", "ECT",
        "rTMS", "tDCS"
    ],
    "rel": [
        "treatment for", "treatment to", "treatment to",
        "treatment for", "treatment for", "treatment for", "treatment for",
        "treatment for", "support for", "treatment for",
        "treatment for", "treatment to", "education for", "adjunct treatment for", "most effective treatment for",
        "effective for", "effective for"
    ],
    "obj": [
        "depression", "increase positive interactions", "reduce depressive symptoms and prevent relapses",
        "mild to moderate major depression", "patients failing classical CBT", "depression", "mild depressive symptoms",
        "depression with marital and family problems", "depressed patients", "acute phase of mdd",
        "subthreshold depression", "reduce depression relapse", "improve treatment adherence and psychosocial function", "depression symptoms",
        "resistant depression", "depression symptoms", "depressive symptoms"
    ]
}




data10 = {
    "subj": [
        "VNS", "tDCS", "DBS", "Magnetic Seizure Therapy", "Phototherapy", "ECT", "rTMS",
        "tDCS", "Antidepressants", "SSRIs", "Paroxetine", "Esctalopram", "Duloxetine", "Venlafaxine", "SSRIs", "Venlafaxine"
    ],
    "rel": [
        "treatment for", "treatment for", "effective for", "effective for", "effective for", "effective for", 
        "treatment for", "treatment to", "treatment for", "compared with", "less effective than", "effective for", 
        "effective for", "compared with", "treatment for", "effective for"
    ],
    "obj": [
        "Resistant Depression", "Depression", "60% of resistant depression patients", "Depressive Symptoms",
        "Seasonal and Non-seasonal Depression", "Most documented and effective method", "Depression", 
        "Reduce Depression", "Depression", "Tricyclic Antidepressants", "Tricyclic Antidepressants", "Major Depressive Disorder", 
        "Major Depressive Disorder", "Other Antidepressants and Placebo", "Depression", "Major Depressive Disorder"
    ]
}




data11 = {
    'subj': ['Selective Serotonin Reuptake Inhibitors (SSRIs)', 'Monoamine Oxidase Inhibitors (MAOIs)', 'Tricyclic Antidepressants (TCAs)', 'Electroconvulsive Therapy (ECT)', 'Venlafaxine', 'Duloxetine', 'Trazodone', 'Nefazodone', 'Bupropion', 'MAOIs', 'TCAs', 'SSRIs', 'Venlafaxine', 'Trazodone', 'Nefazodone', 'Bupropion'],
    'rel': ['effective for', 'effective for', 'effective for', 'effective for', 'more effective than', 'effective for', 'used as', 'effective for', 'effective for', 'not effective for', 'more effective than', 'effective for', 'comparable to', 'used as', 'associated with', 'effective for'],
    'obj': ['Depression', 'Depression with atypical features', 'Major Depression', 'Depression in specific situations', 'SSRIs in some cases', 'Depression', 'Sedative-hypnotic more than an antidepressant', 'Depression', 'Depression with low to moderate anxiety levels', 'Depression in patients resistant to multiple trials with SSRIs and SNRIs', 'SSRIs for hospitalized patients', 'Depression', 'TCAs', 'Sedative-hypnotic', 'Rare cases of clinical idiosyncratic hepatotoxicity', 'Depression'],
}




data12 = {
    "subj": [
        "Mirtazapine",
        "Mianserin",
        "Vilazodone",
        "Vortioxetine",
        "SSRIs",
        "SSRIs",
        "Ketamine",
        "Esketamine",
        "Citalopram", 
        "Escitalopram", 
        "Fluoxetine", 
        "Sertraline", 
        "Vortioxetine", 
        "Amitriptyline", 
        "Clomipramine", 
        "Duloxetine", 
        "Fluvoxamine", 
        "Trazodone", 
        "Venlafaxine",
        "MAOIs",
        "Cognitive Behavioral Therapy (CBT)",
        "Interpersonal Therapy (IPT)",
        "Ketamine",
        "Ketamine",
        "Esketamine",
        "NMDA receptor antagonist",
        "D-cycloserine",
        "Rapastinel"
    ],
    "rel": [
        "example of",
        "example of",
        "less side effects than",
        "effective for",
        "effective for",
        "as effective as",
        "effective for",
        "approved for",
        "more tolerable than",
        "more tolerable than",
        "more tolerable than",
        "more tolerable than",
        "more tolerable than",
        "less tolerable than",
        "less tolerable than",
        "less tolerable than",
        "less tolerable than",
        "less tolerable than",
        "less tolerable than",
        "side effects of",
        "treatment for",
        "treatment for",
        "treatment to",
        "side effects of",
        "side effects of",
        "example of",
        "example of",
        "example of"
    ],
    "obj": [
        "Antidepressant",
        "Antidepressant",
        "SSRIs",
        "Depression-related cognitive impairment",
        "Depression",
        "Mirtazapine",
        "Treatment-resistant depression",
        "Treatment-resistant depression",
        "Other antidepressants",
        "Other antidepressants",
        "Other antidepressants",
        "Other antidepressants",
        "Other antidepressants",
        "Antidepressants",
        "Antidepressants",
        "Antidepressants",
        "Antidepressants",
        "Antidepressants",
        "Antidepressants",
        "Nausea, vomiting, sexual dysfunction, sedation, priapism, cardiotoxicity",
        "Depression",
        "Depression",
        "Reduce depression symptoms",
        "Nausea, vomiting, dizziness, neurotoxicity, cognitive dysfunction",
        "Abuse risk, lack of long-term studies",
        "Antidepressant",
        "Antidepressant",
        "Antidepressant"
    ]
}




data13= {
    "subj": ["CBT", "Behavioral Activation", "Acceptance and Commitment Therapy", "CCBT", "Schema Therapy", 
             "IPT", "PST", "Marital and Family Therapy", "ST", "Psychodynamic Therapy", "Group Therapy", "MBCT",
             "Psycho-Education", "Physical Exercise", "ECT", "rTMS", "tDCS", 
             "Psychotherapy", "Somatic Treatment"],
    "rel": ["treatment for", "treatment to", "effective for", "option for", "treatment for", 
            "treatment for", "comparable to", "effective for", "used to", 
            "effective in", "reduce", "reduce risk of", 
            "improve", "recommended to", "most effective treatment for", "more effective than", 
            "less effective than", "improve quality of life", "manage"],
    "obj": ["depression", "increase positive interactions", "reduce depressive symptoms", 
            "self-motivated patients with mild to moderate depression", "depression in patients with personality disorder comorbidity",  
            "depression", "other psychotherapeutic methods", "some aspects of depression", 
            "support depressed patients", "acute phase of MDD", "depressive symptoms", 
            "depressive recurrence", "clinical course and treatment adherence", 
            "alleviate symptoms", "resistant depression", "placebo", "antidepressants and rTMS", 
            "depression", "depression via different techniques"]
}




data14 = {
    "subj": [
        "tDCS", 
        "VNS", 
        "DBS", 
        "Magnetic seizure therapy",
        "Phototherapy",
        "ECT",
        "rTMS", 
        "VNS", 
        "VNS", 
        "SSRIs", 
        "Paroxetine",
        "Venlafaxine",
        "Duloxetine",
        "Fluoxetine",
        "Venlafaxine vs SSRIs"
    ],
    "rel": [
        "treatment for", 
        "treatment for", 
        "effective for", 
        "alternative to",
        "effective for",
        "most documented and effective method",
        "well-tolerate profile",
        "side effect of",
        "not effective for",
        "treatment for",
        "effective for",
        "comparison as assess by",
        "comparison as assess by",
        "effective for",
        "more effective than"
    ],
    "obj": [
        "depression",
        "resistant unilateral or bipolar depression", 
        "60% resistant depression patient response",
        "ECT to treat psychiatric disorder",
        "seasonal and non-seasonal depression",
        "somatic methods for depression",
        "depression",
        "potential adverse cardiac effect",
        "typical medical treatment alone",
        "depression",
        "major depression",
        "remission rates in major depressive disorder",
        "remission rates in major depressive disorder",
        "depression",
        "SSRIs"
    ]
}




data15 = {
    "subj": ["Citalopram", "Reboxetine", "Bupropion", "Venlafaxine", "Trazodone", "Nefazodone", "Vortioxetine", 
             "SSRIs", "Mirtazapine", "Ketamine", "Esketamine", "Riluzole", "Psychotherapy", "CBT", "Interpersonal psychotherapy"],
    "rel": ["compared with", "compared with", "example of", "compared with", "compared with", "side effects of", "effective for",
            "effective for", "compared with", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for"],
    "obj": ["Reboxetine", "Citalopram", "Antidepressant", "Placebo", "SSRIs", "Clinical hepatotoxicity", "Major depressive disorder",
            "Depression", "SSRIs", "Treatment-resistant depression", "Treatment-resistant depression", "Moderate-to-severe major depressive disorder", "Depression", "Anxiety and depression", "Depression"]
}




data16 = {
  "subj": ["Monoamine Oxidase Inhibitor (MAOI)", "Tricyclic Antidepressants (TCAs)", "Selective Serotonin Reuptake Inhibitors (SSRIs)", "Serotonin-Norepinephrine Reuptake Inhibitors (SNRIs)", "Venlafaxine", "Duloxetine", "Reboxetine", "Trazodone", "Nefazodone", "Bupropion"],
  "rel": ["effective for", "effective for", "effective for", "recommended for", "effective for", "effective for", "less effective than", "example of", "effective for", "effective for"],
  "obj": ["Depression with atypical features", "Major Depression", "Depression", "SSRI non-responders", "Major Depression", "SSRIs", "SSRIs", "Other Antidepressants", "SSRIs", "Major Depression"],
}




data17={
    "subj": [
        "Mirtazapine", "Mianserin", "Vilazodone", "Vortioxetine",
        "SSRIs", "Citalopram", "Escitalopram", "Fluoxetine",
        "Sertraline", "Vortioxetine", "Amitriptyline", "Clomipramine",
        "Duloxetine", "Fluvoxamine", "Trazodone", "Venlafaxine",
        "Ketamine", "Esketamine", "Cognitive-Behavioral Therapy (CBT)",
        "Interpersonal Therapy (IPT)", "Psychoeducation", "Mindfulness-Based Cognitive Therapy (MBCT)",
        "Cognitive Behavioral Analysis System of Psychotherapy (CBASP)"
    ],
    "rel": [
        "example of", "example of", "less sexual dysfunction than", "effective for",
        "effective for", "more tolerable than", "more tolerable than", "more tolerable than",
        "more tolerable than", "more tolerable than", "side effects of", "side effects of",
        "side effects of", "side effects of", "side effects of", "side effects of",
        "treatment for", "treatment for", "treatment for", "treatment for",
        "recommend for", "recommend for", "recommend for"
    ],
    "obj": [
        "Antidepressant", "Antidepressant", "SSRIs", "Depression-related cognitive impairment",
        "Depression", "Other Antidepressants", "Other Antidepressants", "Other Antidepressants",
        "Other Antidepressants", "Other Antidepressants", "High dropout rates", "High dropout rates",
        "High dropout rates", "High dropout rates", "High dropout rates", "High dropout rates",
        "Treatment-resistant depression", "Treatment-resistant depression", "Depression", "Depression",
        "Depression maintenance", "Depression maintenance", "Chronic depression"
    ]
}




data18={
    "subj": [
        "CBT", "Behavioral Activation", "Acceptance and Commitment Therapy", "Computerized CBT (CCBT)", 
        "Schema Therapy", "Problem-Solve Therapy (PST)", "Marital and Family Therapy (MFT)",
        "Supportive Therapy (ST)", "Psychodynamic therapy", "Group Therapy", "Mindfulness Base Cognitive Therapy (MBCT)",
        "Physical Exercise", "ECT", "RTMS", "TDCS"
    ], 
    "rel": [
        "treatment for", "treatment to", "treatment to", "effective for", 
        "effective for", "effective for", "effective for",
        "effective for",  "effective for", "effective for", "alternative for",
        "recommendation for", "treatment for", "effective for", "less effective than"
    ], 
    "obj": [
        "Depression", "Increase positive interaction", "Reduce depressive symptoms", "Mild to moderate major depression",
        "Depression for patients with personality disorder comorbidity", "Mild depressive symptoms", "Depression associated with marital and family problems",
        "Depression", "Depression related to intrapsychic conflicts", "Subthreshold depression", 
        "Reducing antidepressant treatment without increasing depressive recurrence risk for patients at high risk of relapse",
        "Alleviate symptoms and prevent relapses of depression", "Resistant depression", "Depression", "Antidepressants and RTMS for depression"
    ]
}




data19={
    "subj": ["tdcs", "VNS", "VNS", "VNS", "DBS", "DBS", "Magnetic Seizure Therapy", "Phototherapy", "Phototherapy", "ECT", "rTMS", "tdcs", "Antidepressant", "Antidepressant", "Antidepressant", "Antidepressant", "Antidepressant", "Antidepressant", "Antidepressant", "Antidepressant", "Antidepressant"],
    "rel": ["effective for", "treatment for", "effective for", "not effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "most effective for", "effective for", "varied results for", "effective for", "less effective than", "more effective than", "effective for", "effective for", "not effective for", "effective for", "effective for", "effective for"],
    "obj": ["delayed depression symptoms", "resistant unilateral or bipolar depression", "resistant depression", "typical medical treatment alone", "resistant depression", "anhedonia", "depressive symptoms", "seasonal and non-seasonal depression", "depressive symptoms", "depressive disorder", "depression", "depression", "gamma-aminobutyric acid regulation", "tricyclic antidepressants", "placebo", "major depressive disorder", "depression", "certain depression forms", "depression", "major depression", "major depression"]
}




data20={
    "subj": [
        "Citalopram",
        "Reboxetine",
        "Bupropion",
        "Venlafaxine",
        "Trazodone",
        "Nefazodone",
        "Vortioxetine",
        "Selective serotonin reuptake inhibitors (SSRIs)",
        "Mirtazapine",
        "Ketamine",
        "Esketamine",
        "Riluzole"
    ],
    "rel": [
        "compared with",
        "compared with",
        "example of",
        "effective for",
        "effective for",
        "not effective for",
        "effective for",
        "effective for",
        "compared with",
        "effective for",
        "effective for",
        "effective for"
    ],
    "obj": [
        "Reboxetine",
        "Citalopram",
        "antidepressant",
        "Major depression",
        "Major depressive disorder with high level of anxiety",
        "Hepatotoxicity",
        "Major depressive disorder",
        "Depression",
        "Selective serotonin reuptake inhibitors (SSRIs)",
        "Treatment-resistant depression",
        "Treatment-resistant depression",
        "Moderate-to-severe major depressive disorder"
    ]
}






data21={
    "subj": ["Psychotherapy", "SSRIs", "MAOIs", "TCAs", "ECT", "Fluoxetine", "Venlafaxine", "Duloxetine", "Reboxetine", "Trazodone", "Nefazodone", "Bupropion"],
    "rel": ["treatment for", "example of", "example of", "example of", "treatment for", "effective for", "effective for", "effective for", "less effective than", "effective for", "effective for", "effective for"],
    "obj": ["Depression", "Antidepressant", "Antidepressant", "Antidepressant", "Depression", "Depression", "Depression", "Depression", "SSRIs", "Depression", "Depression", "Depression"]
}




data22= {
    "subj": ["Mirtazapine and Mianserin", "Mirtazapine", "Vilazodone and Vortioxetine", "Vortioxetine", "Citalopram", "Escitalopram", "Fluoxetine", "Sertraline", "Vortioxetine", "Amitriptyline", "Clomipramine", "Duloxetine", "Fluvoxamine", "Trazodone", "Venlafaxine", "Ketamine", "Esketamine", "Psychotherapy", "Cognitive-Behavioral Therapy (CBT)", "Interpersonal Therapy (IPT)"],
    "rel": ["example of", "effective for", "not fully clear", "effective for", "more tolerable than", "more tolerable than", "more tolerable than", "more tolerable than", "effective for", "high dropout rates due to", "high dropout rates due to", "high dropout rates due to", "high dropout rates due to", "high dropout rates due to", "high dropout rates due to", "effective for", "treatment for", "treatment for", "treatment for", "treatment for"],
    "obj": ["Antidepressant", "Depression", "SSRIs", "Depression-related cognitive impairment", "Other antidepressants", "Other antidepressants", "Other antidepressants", "Other antidepressants", "Depression", "Severe side effects", "Severe side effects", "Severe side effects", "Severe side effects", "Severe side effects", "Severe side effects", "Resistant depression and acute suicidal ideation", "Treatment-resistant depression", "Depression", "Depression", "Depression"]
}



data23={
    "subj": ["CBT", "IPT", "Behavioral Activation", "Acceptance and Commitment Therapy", "Computerized CBT", "Marital and Family Therapy", "Supportive Therapy", "Psychodynamic Therapy", "Group Therapy", "Physical Exercise", "Psycho-education", "ECT", "RTMS and Antidepressants", "TDCS"],
    "rel": ["treatment for", "treatment for", "treatment to", "effective for", "effective for", "effective for", "treatment for", "treatment for", "effective for", "recommendation for", "treatment to", "treatment for", "more effective than", "treatment for"],
    "obj": ["depression", "mild to moderate major depressive episodes in adults", "reduce depression", "reduce depressive symptoms", "mild to moderate major depression or complementary to pharmacotherapy", "some aspects of depression", "depression", "depression", "post-depressive-symptomatology treatment", "alleviate symptoms and prevent relapses", "understand depression management", "resistant depression", "placebo condition", "depressive symptoms in unipolar or bipolar depression patients"]
}




data24={
    "subj": [
        "tDCS", "VNS", "DBS", "Magnetic Seizure Therapy", "Phototherapy", "ECT", 
        "rTMS", "Antidepressants", "Psychotherapy", "Somatic Therapy", "VNS", 
        "VNS", "DBS", "Phototherapy", "Phototherapy"
    ],
    "rel": [
        "suitable for", "treatment for", "effective for", "alternative to", "effective for", 
        "most effective method", "well-tolerated profile", "under investigation for", 
        "ongoing simplification for", "confirmed indication for", "less effective than", 
        "demonstrate improvement in", "quick impact on", "relation to", "benefit of"
    ],
    "obj": [
        "remote supervised treatment", "depression resistant to other treatments", "resistant depression", 
        "ECT", "seasonal and non-seasonal depression", "neuro-modulation treatment", 
        "depression treatment", "new molecule in gamma-aminobutyric acid regulation and glutamate transmission",
        "depression treatment", "certain depression forms", "medical treatment alone", 
        "depressive symptoms over 6 to 12 months", "anhedonia", "sleep deprivation and drug treatment", 
        "seasonal and non-seasonal depression"
    ]
}




{
    "subj": ["Bupropion", "Citalopram", "Reboxetine", "Venlafaxine", "Trazodone", "Nefazodone", "Ketamine", "Esketamine", "Vortioxetine", "Mirtazapine", "Vilazodone", "Levomilnacipran", "Behavioral Activation Treatment", "IPT", "Cognitive Behavioral Therapy"],
    "rel": ["effective for", "effective for", "not effective for", "effective for", "effective for", "side effects of", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "treatment for", "treatment for", "treatment for"],
    "obj": ["depression with high anxiety", "major depressive disorder", "major depressive disorder", "major depression", "major depressive disorder", "hepatotoxicity", "treatment-resistant depression", "treatment-resistant depression", "major depressive disorder", "major depressive disorder", "major depressive disorder", "major depressive disorder", "depression", "depression", "depression"]
}




{
    "subj": ["Psychoeducation", "Exercise", "Psychotherapy", "Electroconvulsive Therapy", "RTMS", "TDCS", "Vagus Nerve Stimulation", "Deep Brain Stimulation", "Magnetic Seizure Therapy", "Light Therapy", "Sleep Deprivation"],
    "rel": ["effective for"] * 11,
    "obj": ["Depression"] * 11
}






{
    "subj": [
        "Mirtazapine", "Escitalopram", "Venlafaxine", "Sertraline",
        "Duloxetine", "Fluoxetine", "Fluvoxamine", "Paroxetine", "Reboxetine",
        "Escitalopram", "Sertraline", "Sertraline"
    ],
    "rel": [
        "more effective than", "more effective than", "more effective than", "more effective than",
        "less effective than", "less effective than", "less effective than", "less effective than", "less effective than",
        "has better acceptability than", "has better acceptability than", "treatment for"
    ],
    "obj": [
        "Duloxetine/Fluoxetine/Fluvoxamine/Paroxetine/Reboxetine", "Duloxetine/Fluoxetine/Fluvoxamine/Paroxetine/Reboxetine", "Duloxetine/Fluoxetine/Fluvoxamine/Paroxetine/Reboxetine", "Duloxetine/Fluoxetine/Fluvoxamine/Paroxetine/Reboxetine",
        "All other tested antidepressants", "All other tested antidepressants", "All other tested antidepressants", "All other tested antidepressants", "All other tested antidepressants",
        "Duloxetine/Fluvoxamine/Paroxetine/Reboxetine/Venlafaxine", "Duloxetine/Fluvoxamine/Paroxetine/Reboxetine/Venlafaxine", "Adults Moderate to Severe Major Depression"
    ]
}




{
    "subj": ["Mirtazapine", "Escitalopram", "Venlafaxine", "Sertraline", "Reboxetine",
             "Escitalopram", "Sertraline", "Sertraline", "Fluoxetine"],
    "rel": ["more effective than", "more effective than", "more effective than", "more effective than", "less effective than",
            "effective for", "effective for", "treatment", "example of"],
    "obj": ["Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine",
            "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine",
            "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine",
            "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine",
            "All other antidepressants tested",
            "Depression", "Depression", "Adults Moderate to severe major depression", "Antidepressant"]
}




{
    "subj": [
        "Escitalopram", "Citalopram", "Reboxetine", "Paroxetine",
        "Mirtazapine", "Fluoxetine", "Venlafaxine", "Sertraline",
        "Duloxetine", "Fluvoxamine", "Mirtazapine", "Fluoxetine",
        "Duloxetine", "Paroxetine", "Fluvoxamine", "Venlafaxine",
        "Reboxetine", "Escitalopram", "Sertraline"
    ],
    "rel": [
        "more effective than", "more effective than", "less effective than", "less effective than",
        "more effective than", "less effective than", "more effective than", "more effective than",
        "less well tolerate than", "less well tolerate than", "more efficacious than", "less efficacious than",
        "less well tolerate than", "less well tolerate than", "less well tolerate than", "less well tolerate than",
        "less well tolerate than", "well tolerate than", "well tolerate than"
    ],
    "obj": [
        "Citalopram", "Reboxetine, Paroxetine", "All other 11 antidepressants", "Fluvoxamine",
        "Fluoxetine, Venlafaxine", "Venlafaxine, Fluvoxamine", "Fluoxetine, Fluvoxamine", "Fluoxetine",
        "Escitalopram, Sertraline", "Citalopram, Escitalopram, Sertraline", "Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", "Many antidepressants",
        "Escitalopram, Sertraline", "Escitalopram, Sertraline", "Citalopram, Sertraline", "Escitalopram",
        "Many other antidepressants", "Duloxetine, Fluvoxamine, Paroxetine, Reboxetine", "Duloxetine, Fluvoxamine, Paroxetine, Reboxetine"
    ]
}




{
    "subj": [
        "Mirtazapine", "Escitalopram", "Venlafaxine", "Sertraline",
        "Duloxetine", "Fluoxetine", "Fluvoxamine", "Paroxetine", "Reboxetine",
        "Escitalopram", "Sertraline", "Reboxetine", "Sertraline"
    ],
    "rel": [
        "more effective than", "more effective than", "more effective than", "more effective than",
        "less effective than", "less effective than", "less effective than", "less effective than", "less effective than",
        "effective for", "effective for", "not effective for", "treatment"
    ],
    "obj": [
        "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", 
        "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", 
        "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", 
        "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine",
        "All other tested antidepressants",
        "All other tested antidepressants",
        "All other tested antidepressants",
        "All other tested antidepressants",
        "All other tested antidepressants",
        "Depression",
        "Depression",
        "Depression",
        "Adults Moderate to severe major depression"
    ]
}




{
    "subj": ["Escitalopram", "Citalopram", "Reboxetine", "Mirtazapine", "Sertraline", "Venlafaxine", "Duloxetine", "Fluoxetine", "Fluvoxamine", "Paroxetine", "Sertraline", "Venlafaxine", "Reboxetine", "Duloxetine", "Paroxetine", "Fluvoxamine", "Mirtazapine", "Escitalopram", "Venlafaxine"],
    "rel": [
        "more effective than", "more effective than", "less effective than", "more effective than", "more effective than", "more effective than",
        "less well tolerate than", "less well tolerate than", "less well tolerate than", "less well tolerate than", "well tolerate than",
        "well tolerate than", "less effective than", "less well tolerate than", "less well tolerate than", "less well tolerate than",
        "more effective than", "well tolerate than", "well tolerate than"
    ],
    "obj": [
        "Citalopram", "Reboxetine and Paroxetine", "all other antidepressants", "Fluoxetine and Venlafaxine", "Fluoxetine", "Fluoxetine and Fluvoxamine",
        "Escitalopram and Sertraline", "Escitalopram and Sertraline", "Citalopram, Escitalopram, Sertraline", "Escitalopram", "Duloxetine, Fluvoxamine, Paroxetine, Reboxetine",
        "Duloxetine, Fluvoxamine, Paroxetine, Reboxetine", "", "Escitalopram and Sertraline", "Escitalopram and Sertraline", "Citalopram, Escitalopram, Sertraline",
        "Fluoxetine", "Duloxetine, Fluvoxamine, Paroxetine, Reboxetine", "Duloxetine, Fluvoxamine, Paroxetine, Reboxetine"
    ]
}




{
    "subj": ["Mirtazapine", "Escitalopram", "Venlafaxine", "Sertraline", "Escitalopram", "Sertraline", "Bupropion", "Citalopram", "Reboxetine", "Sertraline", "Escitalopram", "Sertraline"],
    "rel": ["more effective than", "more effective than", "more effective than", "more effective than", 
            "well tolerated than", "well tolerated than", "well tolerated than", "well tolerated than", 
            "less effective and less acceptable for", "good choice for", "good choice for", "better cost alternative to"],
    "obj": ["Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", 
            "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", 
            "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", 
            "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", 
            "other new-generation antidepressants", "other new-generation antidepressants", 
            "other new-generation antidepressants", "other new-generation antidepressants", 
            "all other 11 drugs", "moderate to severe major depression", 
            "moderate to severe major depression", "Escitalopram"]
}




{
    "subj": [
        "Mirtazapine", "Escitalopram", "Venlafaxine", "Sertraline",
        "Duloxetine", "Fluoxetine", "Fluvoxamine", "Paroxetine", "Reboxetine",
        "Escitalopram", "Sertraline", "Sertraline"
    ],
    "rel": [
        "more effective than", "more effective than", "more effective than", "more effective than",
        "less effective than", "less effective than", "less effective than", "less effective than", "less effective than",
        "treatment to", "treatment to", "treatment"
    ],
    "obj": [
        "Duloxetine/Fluoxetine/Fluvoxamine/Paroxetine/Reboxetine", 
        "Duloxetine/Fluoxetine/Fluvoxamine/Paroxetine/Reboxetine", 
        "Duloxetine/Fluoxetine/Fluvoxamine/Paroxetine/Reboxetine", 
        "Duloxetine/Fluoxetine/Fluvoxamine/Paroxetine/Reboxetine", 
        "All other tested antidepressants", "All other tested antidepressants", 
        "All other tested antidepressants", "All other tested antidepressants", 
        "All other tested antidepressants", 
        "reduce depression", "reduce depression", 
        "Adults Moderate to severe major depression"
    ]
}




[
    {"subj": "Escitalopram", "rel": "more effective than", "obj": "Citalopram"},
    {"subj": "Citalopram", "rel": "more effective than", "obj": "Reboxetine"},
    {"subj": "Citalopram", "rel": "more effective than", "obj": "Paroxetine"},
    {"subj": "Mirtazapine", "rel": "more effective than", "obj": "Fluoxetine"},
    {"subj": "Mirtazapine", "rel": "more effective than", "obj": "Venlafaxine"},
    {"subj": "Sertraline", "rel": "more effective than", "obj": "Fluoxetine"},
    {"subj": "Venlafaxine", "rel": "more effective than", "obj": "Fluoxetine"},
    {"subj": "Venlafaxine", "rel": "more effective than", "obj": "Fluvoxamine"},
    {"subj": "Reboxetine", "rel": "less effective than", "obj": "Other antidepressants"},
    {"subj": "Duloxetine", "rel": "less tolerated than", "obj": "Escitalopram"},
    {"subj": "Paroxetine", "rel": "less tolerated than", "obj": "Sertraline"},
    {"subj": "Fluvoxamine", "rel": "less tolerated than", "obj": "Citalopram"},
    {"subj": "Venlafaxine", "rel": "less tolerated than", "obj": "Escitalopram"},
    {"subj": "Mirtazapine", "rel": "example of", "obj": "Antidepressant"},
    {"subj": "Escitalopram", "rel": "example of", "obj": "Antidepressant"},
    {"subj": "Sertraline", "rel": "example of", "obj": "Antidepressant"},
    {"subj": "Venlafaxine", "rel": "example of", "obj": "Antidepressant"},
    {"subj": "Fluoxetine", "rel": "example of", "obj": "Antidepressant"}
]




{
    "subj": [
        "Mirtazapine", "Escitalopram", "Venlafaxine", "Sertraline", 
        "Escitalopram", "Sertraline", "Bupropion", "Citalopram",
        "Reboxetine", "Fluvoxamine", "Paroxetine", "Duloxetine",
        "Reboxetine", "Sertraline"
    ],
    "rel": [
        "more effective than", "more effective than", "more effective than", "more effective than",
        "better tolerated than", "better tolerated than", "better tolerated than", "better tolerated than",
        "least effective", "least effective", "least effective", "least effective",
        "least tolerated", "good balance for"
    ],
    "obj": [
        "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine",
        "other new-generation antidepressants", "other new-generation antidepressants", "other new-generation antidepressants", "other new-generation antidepressants",
        "Duloxetine, Fluvoxamine, Paroxetine", "Paroxetine, Reboxetine", "Duloxetine, Reboxetine", "Reboxetine",
        "all other drugs", "treatment for moderate to severe major depression"
    ]
}




{
    "subj": ["Venlafaxine", "Sertraline", "Escitalopram", "Paroxetine", "Fluoxetine", "Citalopram", "Mirtazapine", "Duloxetine", "Reboxetine", "Fluvoxamine"],
    "rel": ["effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for"],
    "obj": ["Major Depression", "Major Depression", "Major Depression", "Major Depression", "Major Depression", "Major Depression", "Major Depression", "Major Depression", "Major Depression", "Major Depression"]
}




{
    "subj": ["Mirtazapine", "Escitalopram", "Venlafaxine", "Sertraline", "Reboxetine", "Escitalopram and Sertraline", "Sertraline"],
    "rel": ["more effective than", "more effective than", "more effective than", "more effective than", "less effective than", "side effects of", "treatment for"],
    "obj": ["Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine", "All other antidepressants tested", "Fewer discontinuations", "Moderate to severe major depression in adults"]
}




{
    "subj": ["Escitalopram", "Escitalopram", "Mirtazapine", "Sertraline", "Venlafaxine", "Fluoxetine", "Reboxetine", "Duloxetine", "Paroxetine", "Fluvoxamine", "Escitalopram", "Venlafaxine"],
    "rel": ["more effective than", "more effective than", "more effective than", "more effective than", "more effective than", "less effective than", "less effective than", "less well tolerate than", "less well tolerate than", "less well tolerate than", "well tolerate than", "less well tolerate than"],
    "obj": ["Citalopram", "Fluoxetine", "Fluoxetine", "Fluoxetine", "Fluvoxamine", "almost all studied antidepressants", "all other 11 antidepressants", "Escitalopram", "Escitalopram", "Citalopram", "Duloxetine", "Escitalopram"]
}




[
    {"subj": "Mirtazapine", "rel":"more effective than", "obj": "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine"},
    {"subj": "Escitalopram", "rel":"more effective than", "obj": "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine"},
    {"subj": "Venlafaxine", "rel":"more effective than", "obj": "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine"},
    {"subj": "Sertraline", "rel":"more effective than", "obj": "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine"},
    {"subj": "Escitalopram, Sertraline, Citalopram, Bupropion", "rel":"more acceptable than", "obj": "Other new-generation antidepressants"},
    {"subj": "Reboxetine", "rel":"less effective and acceptable than", "obj": "All other 11 drugs"},
    {"subj": "Sertraline, Escitalopram", "rel":"recommended for", "obj": "Starting treatment for moderate to severe major depression"}
]




{
    "subj": [
        "Venlafaxine", "Sertraline", "Fluoxetine", "Milnacipran",
        "Reboxetine", "Escitalopram", "Duloxetine", "Mirtazapine",
        "Citalopram", "Paroxetine", "Reboxetine", "Bupropion",
        "Venlafaxine XR", "Sertraline", "Fluoxetine"
    ],
    "rel": [
        "more effective than", "more effective than", "treatment", "example of",
        "effective for", "effective for", "effective for", "effective for",
        "effective for", "effective for", "less effective than", "side effects of",
        "treatment for", "treatment for", "side effects of"
    ],
    "obj": [
        "SSRIs", "SSRIs", "depression", "antidepressant",
        "depression", "depression", "depression", "depression",
        "depression", "depression", "Bupropion", "sexual dysfunction",
        "depression", "major depression", "sexual dysfunction"
    ]
}




{
  "subj": ["Extended-release bupropion hydrochloride", "Extended-release venlafaxine hydrochloride", "Placebo", "Wellbutrin (bupropion HCl) sustain release", "Fluoxetine", "Paroxetine", "Duloxetine", "Escitalopram", "Citalopram", "Paroxetine CR", "Sertraline", "LU 26-054 (Escitalopram)"],
  "rel": ["treatment for", "treatment for", "not effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for", "effective for"],
  "obj": ["Major depressive disorder", "Major depressive disorder", "Major depressive disorder", "Moderate to severe recurrent major depression", "Major depressive disorder", "Major depressive disorder", "Acute treatment of major depression", "Treatment of major depressive disorder", "Treatment of major depressive disorder", "Treatment of major depressive disorder with anxiety", "Treatment of major depressive disorder", "Treatment of major depressive disorder"]
}




{
    "subj": ["Major depressive disorder", "Citalopram", "Sustained-release bupropion", "Sertraline", "Extended-release venlafaxine", "Anxious depression", "Anxious symptoms"],
    "rel": ["coexists with", "example of", "example of", "example of", "example of", "associated with", "predictor of"],
    "obj": ["Anxiety", "Antidepressant", "Antidepressant", "Antidepressant", "Antidepressant", "Poorer acute outcome following antidepressant treatment", "Treatment resistance in adults with depression"]
}




{
    "subj": ["Major Depressive Disorder", "Citalopram", "Anxious depression", "Bupropion", "Sertraline", "Venlafaxine", "Anxious depression", "Anxious depression"],
    "rel": ["has symptom", "treatment for", "leads to", "example of", "example of", "example of", "associated with", "impact on"],
    "obj": ["Anxiety", "Major Depressive Disorder", "Poorer treatment outcomes", "Antidepressant", "Antidepressant", "Antidepressant", "More frequent side effects", "Treatment effectiveness"]
}




[
  {"subj": "Citalopram", "rel": "treatment for", "obj": "Major Depressive Disorder"},
  {"subj": "Anxious depression", "rel": "less likely to remit with", "obj": "Citalopram"},
  {"subj": "Nonanxious depression", "rel": "more likely to remit with", "obj": "Citalopram"},
  {"subj": "Anxious depression", "rel": "associated with", "obj": "Higher side effects from Citalopram"},
  {"subj": "Anxious depression", "rel": "associated with", "obj": "More serious adverse events"},
  {"subj": "Bupropion", "rel": "treatment to", "obj": "Reduce depression in STAR*D Level 2"},
  {"subj": "Sertraline", "rel": "treatment to", "obj": "Reduce depression in STAR*D Level 2"},
  {"subj": "Venlafaxine", "rel": "treatment to", "obj": "Reduce depression in STAR*D Level 2"},
  {"subj": "Buspirone", "rel": "treatment to", "obj": "Reduce depression in STAR*D Level 2"},
  {"subj": "Anxious depression", "rel": "has higher frequency, intensity, and burden of", "obj": "Side effects than Nonanxious depression"},
  {"subj": "Anxious depression", "rel": "not effective for", "obj": "Citalopram compared to nonanxious depression"},
  {"subj": "Hospitalization for general medical condition", "rel": "more in", "obj": "Anxious depression group"}
]




{
    "subj": ["Major Depressive Disorder", "Citalopram", "Bupropion", "Sertraline", "Venlafaxine", "Buspirone", "Anxiolytics and Sedative/Hypnotics"],
    "rel": ["associated with", "treatment for", "option for", "option for", "option for", "augmentation with", "use predicts"],
    "obj": ["Anxious Depression", "MDD with or without anxious symptoms", "switching in non-remitters to Citalopram", "switching in non-remitters to Citalopram", "switching in non-remitters to Citalopram", "citalopram for augmentation", "treatment resistance in older adults"]
}




{
    "subj": [
        "Anxious depression", "Citalopram", "Citalopram", "Anxious depression",
        "Sustained-release bupropion", "Sertraline", "Extended-release venlafaxine",
        "Citalopram + Sustained-release bupropion", "Citalopram + Buspirone",
        "Anxious depression", "Citalopram", "Anxious depression"
    ],
    "rel": [
        "more common in", "not effective for", "effective for", "treatment characteristic",
        "example of", "example of", "example of", "treatment for",
        "treatment for", "side effects of", "side effects of", "worse outcome with"
    ],
    "obj": [
        "Certain demographics", "Nonanxious depression", "Anxious depression", "Fewer visits less time",
        "Antidepressant", "Antidepressant", "Antidepressant", "Depression",
        "Depression", "Greater frequency, intensity, burden of side effects", "Hospitalization for general medical condition", "Antidepressant treatment"
    ]
}




{
    'subj': ['Anxious depression', 'Anxious depression', 'Anxious depression', 'Anxious depression treatment', 'Citalopram', 'Side effects', 'Benzodiazepine', 'Antidepressant treatment', 'Antidepressant monotherapy', 'Anxious depression', 'Physical symptoms'],
    'rel': ['symptom of', 'less likely to achieve', 'side effect more severe', 'effective for', 'treatment for', 'side effect of', 'treatment to', 'not effective for', 'not effective for', 'characterized by', 'symptom of'],
    'obj': ['Major Depressive Disorder', 'remission than nonanxious depression', 'than nonanxious depression', 'Anxious depression', 'Major depression', 'Antidepressant', 'reduce anxiety and insomnia', 'Anxious depression with high physical illness burden and low socioeconomic status', 'Anxious depression in STAR*D Level 1', 'High levels of anxiety', 'Anxious depression']
}




{
    "subj": [
        "Major depressive disorder",
        "Anxious depression",
        "Citalopram",
        "Sustained-release bupropion",
        "Sertraline",
        "Extended-release venlafaxine",
        "Sustained-release bupropion",
        "Buspirone",
        "Anxiolytic medications",
        "Sedative/hypnotic medications",
        "Anxious depression",
        "Nonanxious depression"
    ],
    "rel": [
        "symptom of",
        "exhibits",
        "effective for",
        "option for",
        "option for",
        "option for",
        "augmentation option for",
        "augmentation option for",
        "predictor of",
        "predictor of",
        "less likely to achieve remission with",
        "more likely to achieve remission with",
    ],
    "obj": [
        "anxiety, nervousness",
        "higher depression severity, functional impairment",
        "major depressive disorder",
        "switching from citalopram",
        "switching from citalopram",
        "switching from citalopram",
        "citalopram in non-remitters",
        "citalopram in non-remitters",
        "treatment resistance in depression",
        "treatment resistance in depression",
        "antidepressant treatment",
        "antidepressant treatment"
    ]
}




{
    'subj': ['Citalopram', 'Anxious Depression', 'Anxious Depression', 'Anxious Depression', 'Anxious Depression', 'Major Depression with Anxious Features', 'Nonanxious Depression', 'Citalopram', 'Sustained-release Bupropion', 'Sertraline', 'Extended-release Venlafaxine', 'Sustained-release Bupropion for augmentation', 'Buspirone for augmentation'],
    'rel': ['effective for', 'more severe than', 'associated with', 'not effective for', 'has side effects of', 'effective for', 'effective for', 'treatment for', 'example of', 'example of', 'example of', 'example of', 'example of'],
    'obj': ['Major Depression', 'Nonanxious Depression', 'Greater Psychological and Physical Impairment', 'Major Depression', 'Increased Frequency, Intensity, and Burden of Side Effects', 'Major Depression', 'Major Depression', 'Major Depression', 'Antidepressant', 'Antidepressant', 'Antidepressant', 'Antidepressant', 'Antidepressant']
}




{
    "subj": ["Citalopram", "Anxious depression", "Nonanxious depression", "Benzodiazepine"],
    "rel": ["treatment for", "less likely to achieve remission compared to", "side effects of", "treatment to"],
    "obj": ["Depression", "Nonanxious depression", "Anxious depression", "Improve symptoms of anxious depression"]
}




{
    "subj": ["Citalopram", "Bupropion-SR", "Sertraline", "Venlafaxine-XR", "QID-SR", "Cumulative Illness Rating Scale", "Work and Social Adjustment Scale", "QID-C", "STAR*D Study", "Suicide risk management"],
    "rel": ["effective for", "treatment for", "treatment for", "treatment for", "used for", "used for", "measures", "used for", "evaluated", "part of"],
    "obj": ["depression", "depression after SSRI failure", "moderate to severe major depression", "depression after SSRI failure", "measuring depression severity", "assessing medical illness burden", "impairment in functioning due to depression", "clinician rating of depression", "antidepressant comparison", "STAR*D study"]
}
data=[data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12,data13,data14]
for i,_ in enumerate(data):
  data[i]=pd.DataFrame(data[i])
pd.concat(data).reset_index().drop(columns=['index']).to_csv('nlp3LLM.csv')


In [99]:












ubj": "Nefazodone", "rel": "example of", "obj": "antidepressant"},
    {"subj": "Vortioxetine", "rel": "treatment", "obj": "major depressive disorder"},
    {"subj": "Mirtazapine", "rel": "effective for", "obj": "major depressive disorder"},
    {"subj": "Vilazodone", "rel": "example of", "obj": "antidepressant"},
    {"subj": "Levomilnacipran", "rel": "example of", "obj": "antidepressant"},
    {"subj": "Vortioxetine", "rel": "example of", "obj": "antidepressant"},
    {"su
 




data=[data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12,data13,data14,data15,data16,data17,data18,data19,data20,data21,data22,data23,
 data24,data25,data26,data27,data28,data29,data30,data31,data32,data33,data34,data35,data36]
for i,_ in enumerate(data):
  data[i]=pd.DataFrame(data[i])
pd.concat(data).reset_index().drop(columns=['index']).to_csv('nlpLLM.csv')

In [76]:
pd.merge(d1,d2).merge(d3)

,subj,rel,object1,rel2,object2


In [17]:
data = {
    "entity1": ["Mirtazapine", "Escitalopram", "Venlafaxine", "Sertraline", "Sertraline", "Duloxetine", "Fluoxetine", "Fluvoxamine", "Paroxetine", "Reboxetine", "Escitalopram", "Sertraline", "Duloxetine", "Fluvoxamine", "Paroxetine", "Reboxetine", "Escitalopram", "Sertraline", "Reboxetine", "Sertraline"],
    "relationship": ["more efficacious than", "more efficacious than", "more efficacious than", "more efficacious than", "good profile of acceptability", "less well tolerated than", "less well tolerated than", "less well tolerated than", "less well tolerated than", "least efficacious", "less well tolerated than", "less well tolerated than", "less well tolerated than", "less well tolerated than", "less well tolerated than", "less tolerated than", "best choice for moderate to severe major depression", "favorable balance between benefits, acceptability, and cost", "should not be used as first-line treatment", "considered better option due to cost"],
    "entity2": ["Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine", "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine", "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine", "Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine", "Duloxetine, Fluvoxamine, Paroxetine, Reboxetine, Venlafaxine", "Escitalopram, Sertraline", "Escitalopram, Sertraline", "Citalopram, Escitalopram, Sertraline", "Many other antidepressants", "All other tested antidepressants", "Duloxetine, Fluvoxamine, Paroxetine, Reboxetine", "Duloxetine, Fluvoxamine, Paroxetine, Reboxetine", "Escitalopram, Sertraline", "Citalopram, Escitalopram, Sertraline", "Many other antidepressants", "Bupropion, Citalopram, Escitalopram, Fluoxetine, Sertraline", "in adults", "in adults", "for major depression", "due to lower cost in most countries"]
    }

In [61]:
a={"subj": ["Anxious Depression", "Anxious Depression", "Anxious Depression", "Nonanxious Depression", "Citalopram", "Switching Antidepressants", "Switching Antidepressants", "Switching Antidepressants",  "Antidepressant Augmentation", "Antidepressant Augmentation" ],
   "rel": ["association", "association", "association", "association","treatment outcome", "remission rate", "remission rate", "remission rate", "remission rate", "remission rate"],
   "object1": ["Poor treatment outcome", "Greater side effects", "More serious adverse events", "Better treatment outcome","Level 1 Treatment", "Bupropion", "Sertraline", "Extended-release Venlafaxine", "Citalopram plus Bupropion", "Citalopram plus Buspirone"],
   "rel2": ["association with", "association with", "association with", "association with", "association with", "Treatment Option", "Treatment Option", "Treatment Option","Treatment Option", "Treatment Option" ],
   "object2": ["Increased treatment time", "Increased treatment drop-out", "Hospitalizations", "Quicker remission","Anxious vs Nonanxious Depression", "Anxious Depression", "Anxious Depression", "Anxious Depression","Anxious Depression", "Anxious Depression" ]
   }

In [62]:
pd.DataFrame(a)

,subj,rel,object1,rel2,object2
0,Anxious Depression,association,Poor treatment outcome,association with,Increased treatment time
1,Anxious Depression,association,Greater side effects,association with,Increased treatment drop-out
2,Anxious Depression,association,More serious adverse events,association with,Hospitalizations
3,Nonanxious Depression,association,Better treatment outcome,association with,Quicker remission
4,Citalopram,treatment outcome,Level 1 Treatment,association with,Anxious vs Nonanxious Depression
5,Switching Antidepressants,remission rate,Bupropion,Treatment Option,Anxious Depression
6,Switching Antidepressants,remission rate,Sertraline,Treatment Option,Anxious Depression
7,Switching Antidepressants,remission rate,Extended-release Venlafaxine,Treatment Option,Anxious Depression
8,Antidepressant Augmentation,remission rate,Citalopram plus Bupropion,Treatment Option,Anxious Depression
9,Antidepressant Augmentation,remission rate,Citalopram plus Buspirone,Treatment Option,Anxious Depression


In [63]:
print(cand_name)

Based on the information gathered from the clinical paper, a DataFrame named "paper1" has been created to illustrate the entities, their definitions, overarching ideas, and their relationships to each other extracted from the paper Below is a representation of "paper1" DataFrame in a visually friendly table format:

| entity1 | relationship | entity2 |
|---------|--------------|---------|
| Depression | defined by | major depressive disorder |
| Escitalopram, Mirtazapine, Sertraline, Venlafaxine | more efficacious than | Duloxetine, Fluoxetine, Fluvoxamine, Paroxetine, Reboxetine  |
| Duloxetine, Paroxetine | less well tolerated than | Escitalopram, Sertraline  |
| Reboxetine | significantly less efficacious than | other 11 antidepressants  |

Please note, the above table is a simplified extraction focusing on key findings regarding the efficacy and tolerability of specific antidepressants for the treatment of depression, as detailed in the clinical paper


In [60]:
import pandas as pd
data = {
    "entity1": ["Depression", "New-generation antidepressants", "Comparative efficacy", "Major depression",
                "Antidepressants", "Escitalopram, mirtazapine, sertraline, venlafaxine", "Reboxetine",
                "Sertraline", "Duloxetine, paroxetine", "Bupropion, citalopram, fluoxetine, sertraline"],
    "relationship": ["treated by", "include", "evaluated through", "assessed for",
                     "compared in", "more efficacious than", "found to be",
                     "shown to have", "less well tolerated than", "better tolerated than"],
    "entity2": ["12 new-generation antidepressants", "bupropion, citalopram, duloxetine, escitalopram, fluoxetine, fluvoxamine, milnacipran, mirtazapine, paroxetine, reboxetine, sertraline, venlafaxine",
                "multiple-treatments meta-analysis", "acute-phase treatment efficacy and acceptability",
                "randomized controlled trials", "duloxetine, fluoxetine, fluvoxamine, paroxetine, reboxetine", "significantly less efficacious than other 11 antidepressants",
                "less clear benefits compared with escitalopram, venlafaxine, mirtazapine",
                "escitalopram and sertraline", "reboxetine"]
}

paper1 = pd.DataFrame(data)
paper1


,entity1,relationship,entity2
0,Depression,treated by,12 new-generation antidepressants
1,New-generation antidepressants,include,"bupropion, citalopram, duloxetine, escitalopra..."
2,Comparative efficacy,evaluated through,multiple-treatments meta-analysis
3,Major depression,assessed for,acute-phase treatment efficacy and acceptability
4,Antidepressants,compared in,randomized controlled trials
5,"Escitalopram, mirtazapine, sertraline, venlafa...",more efficacious than,"duloxetine, fluoxetine, fluvoxamine, paroxetin..."
6,Reboxetine,found to be,significantly less efficacious than other 11 a...
7,Sertraline,shown to have,less clear benefits compared with escitalopram...
8,"Duloxetine, paroxetine",less well tolerated than,escitalopram and sertraline
9,"Bupropion, citalopram, fluoxetine, sertraline",better tolerated than,reboxetine


In [ ]:
!git branch main

In [ ]:
!pwd

/content


In [ ]:

!git add /content/
!git remote add origin "https://github.com/rishikasrinivas/KnowledgeGraphMentalHealth.git"
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"
!git commit -m "GPT code w/ trial kg"
!git push -u origin main

fatal: pathspec 'Brightside' did not match any files
error: remote origin already exists.
On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	KnowledgeGraphMentalHealth/

nothing added to commit but untracked files present (use "git add" to track)
fatal: could not read Username for 'https://github.com': No such device or address
